# Cuotas vs Contado vs Mixto

## Motivacion
Actualmente los bancos ofrecen cierta financiación para compras con tarjetas en un cierto número de cuotas, la duda que se intenta resolver
con este ejercicio es saber si teniendo el total del efectivo para pagar
un producto determinado ¿conviene pagar de contado en un solo pago o pagar en cuotas y hacer como si pagaramos de contado pero la diferencia colocarla en un plazo fijo y comparar el rendimiento?

Esta duda surgio en base a la siguiente noticia probar su veracidad y permitir aplicarla:

[El truco para pagar la mitad por cualquier celular: se hace aprovechando la crisis económica](https://www.cronista.com/infotechnology/gadgets/el-rulo-que-usa-las-tasa-de-los-bancos-y-las-cuotas-para-pagar-la-mitad-por-cualquier-celular/)

La idea en si es generalizar no solo para caso cuotas sin interes sino cualquier escenario posible que nos permita tomar una desicion.

In [30]:
import pandas as pd
from decimal import Decimal
import datetime
from dateutil.relativedelta import relativedelta

## Funciones de calculo de tazas y rendimientos contable

In [21]:
def calcular_rendimiento(C, r, t=1, n=12):
    """
    C: capital inicial,
    r: tasa de interes expresada en decimales
    t: cantidad de años que se invierte
    n: numero de periodos
    """
    
    return C * (1 + (r/n)) ** (n*t)

def calcular_rendimiento_simplificado(C, r, d=1):
    """
    calculo usando una aproximacion de la tasa del simulador de BNA
    https://www.bna.com.ar/SimuladorPlazoFijo/SubInterna/PlazoFijo?subInterna=SimuladorPlazoFijo&id=PFElectronico
    
    C: capital inicial,
    r: tasa de interes expresada en decimales
    d: dias
    """
    t_ = d / 365
    return calcular_rendimiento(C, r, t_, 12)

In [22]:
#test invertir 10.000 a un año un solo periodo
calcular_rendimiento(10000, 0.75, t=1, n=1)

17500.0

In [23]:
#test invertir 10.000 a 12 periodos
calcular_rendimiento(10000, 0.75, t=1, n=12)

20698.899917795225

In [24]:
#test invertir 10.000 a 30 dias usando formula BNA
calcular_rendimiento_simplificado(10000, 0.75, d=30)

10616.17987381662

## Inicializar valores en los siguientes

In [80]:
#valor si lo pagaras de contado
valor_contado = 138713.45

#valor de la cuota fija mensula
cuota = 16047.24  #modificar

# cantidad de cuotas
periodos = 12 #meses

valor_total_tarjeta = cuota * periodos

#Taza anual actual del PF
taza_anual_pf = 0.75

#fecha que realizas la compra y los PF
fecha_compra = "20/10/2022"

# dia del mes que hay que pagar la tarjeta
dia_vence_cuota = 5 #ejemplo 5 de cada mes

## El calculo en si

In [81]:
proxima_cuota = datetime.datetime.strptime(fecha_compra, "%d/%m/%Y").date() + relativedelta(months=1)
proxima_cuota = proxima_cuota.replace(day=dia_vence_cuota) #cuota inicial
ultima_cuota = proxima_cuota + relativedelta(months=periodos - 1)
data = []
for i in range(1, periodos+1):
    pagos_acumulado = cuota * i
    delta_dias = (ultima_cuota - proxima_cuota).days
    rendimiento = 0
    #los plazo fijo no pueden ser por menos de 30 dias
    if delta_dias >= 30: 
        rendimiento = calcular_rendimiento_simplificado(cuota, taza_anual_pf, d=delta_dias) - cuota
    data.append((
        i,
        proxima_cuota.strftime("%d/%m/%Y"),
        cuota,
        #pagos_acumulado,
        delta_dias,
        rendimiento
    ))
    proxima_cuota += relativedelta(months=1) 
df = pd.DataFrame(data, columns=["id", "Fecha", "Cuota", "Días PF", "Rendimiento"])
df

,id,Fecha,Cuota,Días PF,Rendimiento
0,1,05/11/2022,16047.24,334,15178.571616
1,2,05/12/2022,16047.24,304,13366.176113
2,3,05/01/2023,16047.24,273,11603.807592
3,4,05/02/2023,16047.24,242,9947.035197
4,5,05/03/2023,16047.24,214,8536.087223
5,6,05/04/2023,16047.24,183,7063.123937
6,7,05/05/2023,16047.24,153,5721.761855
7,8,05/06/2023,16047.24,122,4417.424968
8,9,05/07/2023,16047.24,92,3229.623440
9,10,05/08/2023,16047.24,61,2074.608423


In [85]:
rendimiento_pf = df["Rendimiento"].sum()
valor_pf = round(valor_total_tarjeta - rendimiento_pf, 2)
ahorro = round(valor_contado - valor_pf, 2)

print(f"VALOR CONTADO: $ {valor_contado}")
print()
print(f"VALOR TARJETA: $ {valor_total_tarjeta} a {periodos} CUOTAS")
print(f"CANTIDAD CUOTA: {periodos}")
print(f"VALOR CUOTA: $ {cuota}")
print()
print(f"TAZA ANUAL PF: {taza_anual_pf * 100}%")
print(f"VALOR RULO PF: $ {valor_pf}")
print(f"AHORRO: ${ahorro}")

VALOR CONTADO: $ 138713.45

VALOR TARJETA: $ 192566.88 a 12 CUOTAS
CANTIDAD CUOTA: 12
VALOR CUOTA: $ 16047.24

TAZA ANUAL PF: 75.0%
VALOR RULO PF: $ 110439.86
AHORRO: $28273.59
